# Building a Spam Filter with Naive Bayes

For our project, we are going to develop a spam filter for SMS messages using the multinomial Naive Bayes algorithm with at least 80% accuracy. Our goal is to classify new messages based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as ham(non-spam).

To "teach" our algorithm, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

## Exploring the Dataset

In [1]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

sms_spam.shape

(5572, 2)

In [2]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in reality most messages that people receive are ham.

## Training and Test Set

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- the training set will have 4,458 messages (about 80% of the dataset)
- the test set will have 1,114 messages (about 20% of the dataset).

In [4]:
# randomize dataset
random = sms_spam.sample(frac=1, random_state=1)

# set index to split dataset
training_index = round(len(random) * 0.8)

# split dataset
training_set = random[:training_index].reset_index(drop=True)
testing_set = random[training_index:].reset_index(drop=True)

training_set['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [5]:
testing_set['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

After analyzing the training and testing dataset we can see the percentages are fairly close to the original dataset's.

## Letter Case and Punctuation

To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Our end goal with this data cleaning process is to bring our training set to this format:

![image](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

We'll begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

In [6]:
# before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
# after cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the Vocabulary

We are now going to create a list of all of the unique words in our data set, which is called the vocabulary.

In [8]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for message in training_set['SMS']:
    for word in message:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

## The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.

In [9]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index]+=1

word_count = pd.DataFrame(word_counts_per_sms)
word_count.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [10]:
training_set_clean = pd.concat([training_set, word_count], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. Let's first calculate:

- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary

In [11]:
# isolate spam and ham messages
spam = training_set_clean[training_set_clean['Label'] == 'spam']
ham = training_set_clean[training_set_clean['Label'] == 'ham']

# calculate probability
p_spam = len(spam) / len(training_set_clean)
p_ham = len(ham) / len(training_set_clean)

# n_spam
n_words_spam = spam['SMS'].apply(len)
n_spam = n_words_spam.sum()

# n_ham
n_words_ham = ham['SMS'].apply(len)
n_ham = n_words_ham.sum()

# n_vocabulary
n_vocabulary = len(vocabulary)

# laplace smoothing
alpha = 1

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters ![p_spam](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29&mode=inline) and ![p_ham](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29&mode=inline)
Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

In [12]:
# initiate parameters
spam_param = {unique_word:[0] for unique_word in vocabulary}
ham_param = {unique_word:[0] for unique_word in vocabulary}

# calculate parameters
for word in vocabulary:
    n_words_occur_spam = spam[word].sum()
    p_words_occur_spam = (n_words_occur_spam + alpha) / (n_spam + alpha * n_vocabulary)
    spam_param[word] = p_words_occur_spam
    
    n_words_occur_ham = ham[word].sum()
    p_words_occur_ham = (n_words_occur_ham + alpha) / (n_ham + alpha * n_vocabulary)
    ham_param[word] = p_words_occur_ham

## Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- takes in as input a new message (w1, w2, ..., wn)
- calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
  - if P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
  - if P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
  - if P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [13]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_param:
            p_spam_given_message *= spam_param[word]
        if word in ham_param:
            p_ham_given_message *= ham_param[word]
            

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this!')

In [14]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [15]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's accuracy

We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

In [16]:
# create function that returns Ham or Spam
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_param:
            p_spam_given_message *= spam_param[word]
        if word in ham_param:
            p_ham_given_message *= ham_param[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'need human clarification'

In [17]:
testing_set['Predicted'] = testing_set['SMS'].apply(classify_test_set)
testing_set.head()

,Label,SMS,Predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


We will now measure the accuracy of our function.

In [18]:
correct = 0
total = len(testing_set)

for row in testing_set.iterrows():
    row = row[1]
    if row['Label'] == row['Predicted']:
        correct+=1

print('Correct: ', correct)
print('Incorrect: ', total - correct)
print('Accuracy: ', correct / total)

Correct:  1100
Incorrect:  14
Accuracy:  0.9874326750448833


Our program was able to execute with 98% accuracy which is a success in my book.

## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.